### [Transfer Learning in Time Series Analysis](https://medium.com/@kylejones_47003/transfer-learning-in-time-series-analysis-4b7f1d1f4bfd)

> Modern neural networks can learn temporal patterns from one domain and apply them to another, dramatically reducing the data needed for accurate predictions. This transfer of knowledge enables organizations to leverage existing models for new applications, from energy forecasting to healthcare monitoring.

Transfer learning represents a paradigm shift in how we approach time series modeling. Traditional time series analysis requires substantial data from the specific domain of interest.

The application of transfer learning to time series data operates through several key mechanisms. Feature-based transfer learning extracts meaningful representations from source time series data that can be applied to target domains.

Parameter-based transfer learning, alternatively, reuses parts of a trained model’s architecture or parameters, fine-tuning them for the new task.

##### Instance-based Transfer Learning

Instance-based transfer learning selectively uses samples from the source domain to augment learning in the target domain. This approach proves particularly valuable when dealing with rare events or anomalies in time series data.

The key challenge lies in identifying which instances from the source domain remain relevant to the target problem.

##### Deep Transfer Learning for Time Series

Deep learning architectures have dramatically expanded the possibilities for transfer learning in time series analysis. Convolutional Neural Networks (CNNs) and Long Short-Term Memory (LSTM) networks can learn hierarchical representations of temporal patterns that often generalize across domains.

A model initially trained on high-frequency financial data might extract features useful for analyzing medical time series, despite the apparent differences between these domains. The deep learning approach to transfer learning often involves freezing early layers of the network while retraining later layers on the target domain.

In [1]:
!pip install -q numpy pandas matplotlib
!pip install -q scikit-learn tensorflow==2.18.0

In [2]:
import warnings
warnings.filterwarnings('ignore')

#### Basic Setup and Data Preparation

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input

# Helper function to create time series sequences
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:(i + seq_length)])
    return np.array(sequences)

In [4]:
# Load and prepare source domain data (e.g., energy consumption)
source_data = pd.read_csv('../data/energy_consumption.csv')
source_scaler = MinMaxScaler()
source_scaled = source_scaler.fit_transform(source_data[['consumption']])
source_sequences = create_sequences(source_scaled, seq_length=24)

# Load and prepare target domain data (e.g., solar production)
# solar_production.csv
target_data = pd.read_csv('https://raw.githubusercontent.com/patricksheehan/All-Your-Battery-Are-Belong-To-Us/refs/heads/master/solar_production.csv')
target_data.rename(columns={'v':'production'}, inplace=True)
target_scaler = MinMaxScaler()
target_scaled = target_scaler.fit_transform(target_data[['production']])
target_sequences = create_sequences(target_scaled, seq_length=24)

#### Building a Base Model for Source Domain

In [5]:
def create_base_model(sequence_length, n_features=1):
    model = Sequential([
        LSTM(64, input_shape=(sequence_length, n_features), return_sequences=True, name='lstm_1'),
        LSTM(32, name='lstm_2'),
        Dense(16, activation='relu', name='dense_1'),
        Dense(1, name='output')
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Train base model on source domain
source_model = create_base_model(24)
source_model.fit(
    source_sequences[:-1], 
    source_scaled[24:], 
    epochs=50,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.1117 - val_loss: 0.0837
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0853 - val_loss: 0.0834
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0856 - val_loss: 0.0835
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0846 - val_loss: 0.0834
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0841 - val_loss: 0.0842
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0845 - val_loss: 0.0835
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0831 - val_loss: 0.0832
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0851 - val_loss: 0.0833
Epoch 9/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0852 - val_loss: 0.0838
Epoch 10/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0846 - val_loss: 0.0837
Epoch 11/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0848 - val_loss: 0.0834
Epoch 12/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/ste

#### Feature-based Transfer Learning

In [6]:
# Extract features from intermediate layer
def create_feature_extractor(base_model, layer_name='lstm_1'):
    return Model(
        inputs=base_model.input,
        outputs=base_model.get_layer(layer_name).output
    )

feature_extractor = create_feature_extractor(source_model)

AttributeError: The layer sequential has never been called and thus has no defined input.

In [7]:
# Create new model using transferred features
def create_transfer_model(feature_extractor, sequence_length):
    inputs = Input(shape=(sequence_length, 1))
    features = feature_extractor(inputs)
    x = LSTM(16)(features)
    outputs = Dense(1)(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

transfer_model = create_transfer_model(feature_extractor, 24)

NameError: name 'feature_extractor' is not defined

#### Fine-tuning Approach

In [ ]:
def create_fine_tuning_model(base_model, trainable_layers=1):
    # Freeze early layers
    for layer in base_model.layers[:-trainable_layers]:
        layer.trainable = False
    
    return base_model

# Clone source model for fine-tuning
fine_tune_model = tf.keras.models.clone_model(source_model)
fine_tune_model.set_weights(source_model.get_weights())
fine_tune_model = create_fine_tuning_model(fine_tune_model)

# Fine-tune on target domain
fine_tune_model.fit(
    target_sequences[:-1],
    target_scaled[24:],
    epochs=20,
    batch_size=32,
    validation_split=0.2
)

#### Domain Adaptation

In [ ]:
class DomainAdapter:
    def __init__(self, source_scaler, target_scaler):
        self.source_scaler = source_scaler
        self.target_scaler = target_scaler
    
    def adapt_sequence(self, sequence, from_domain='source', to_domain='target'):
        if from_domain == 'source' and to_domain == 'target':
            # Inverse transform to original scale
            sequence_orig = self.source_scaler.inverse_transform(sequence)
            # Transform to target scale
            return self.target_scaler.transform(sequence_orig)
        else:
            sequence_orig = self.target_scaler.inverse_transform(sequence)
            return self.source_scaler.transform(sequence_orig)

# Create and use domain adapter
adapter = DomainAdapter(source_scaler, target_scaler)
adapted_sequences = adapter.adapt_sequence(source_sequences)

#### Evaluation and Comparison

In [ ]:
def evaluate_models(models, test_sequences, test_targets):
    results = {}
    for name, model in models.items():
        predictions = model.predict(test_sequences)
        mse = tf.keras.losses.MSE(test_targets, predictions)
        mae = tf.keras.losses.MAE(test_targets, predictions)
        results[name] = {'MSE': float(mse), 'MAE': float(mae)}
    return pd.DataFrame(results).T

# Compare different approaches
models = {
    'Base Model': source_model,
    'Transfer Learning': transfer_model,
    'Fine-tuned': fine_tune_model
}

results = evaluate_models(
    models,
    target_sequences[-100:],
    target_scaled[-100:]
)
print("\nModel Comparison:")
print(results)

#### Visualization of Results

In [ ]:
import matplotlib.pyplot as plt

def plot_predictions(models, test_sequences, true_values, scaler):
    plt.figure(figsize=(15, 6))
    
    # Plot true values
    plt.plot(scaler.inverse_transform(true_values), 
             label='Actual', linewidth=2)
    
    # Plot predictions from each model
    for name, model in models.items():
        predictions = model.predict(test_sequences)
        plt.plot(scaler.inverse_transform(predictions), 
                label=f'{name} Predictions', linestyle='--')
    
    plt.title('Model Predictions Comparison')
    plt.legend()
    plt.grid(True)
    plt.show()

# Visualize results
plot_predictions(
    models,
    target_sequences[-100:],
    target_scaled[-100:],
    target_scaler
)

#### Best Practices and Implementation Strategies

- First, source and target domains should share meaningful similarities in their temporal patterns or underlying generative processes.
- Second, the transfer learning approach should account for differences in scale, sampling frequency, and noise levels between domains.
- Third, validation strategies must carefully assess whether the transferred knowledge improves or potentially degrades performance in the target domain.